# log_income 전처리


### 활용한 데이터: 
1. KOSIS 지역소득통계(생산/지출/분배) 중 분배 계정 -> 개인 소득(명목) 데이터 2017년~2019년

https://kosis.kr/publication/publicationThema.do?pubcode=GS

(사람들이 주택을 사는 능력을 설명하려면? 얼마나 돈을 벌었고 쓸 수 있는지가 중요 -> 비자발적 지출을 제외한 소득으로, 실제 소비/저축/투자에 쓸 수 있는 금액)

2. 전국 시군구 단위 연소득 평균 (2024년도 자료)

https://www.bigdata-culture.kr/bigdata/user/data_market/detail.do?id=75ab3e79-6f9b-4d80-934b-07746d384096

-> 시군구별 비중 정보 활용 (ex.서울특별시 평균 대비 강남구 비중)



In [ ]:
# run once.
! pip install pandas
! pip install openpyxl

In [ ]:
import pandas as pd

# 1. 다중 헤더 읽기 (4~5행 → index 3,4)
df = pd.read_excel('13. 개인소득(명목).xlsx', header=[3, 4])

# 2. 한글 지역명만 열 이름으로 사용
df.columns = df.columns.get_level_values(0)

# 3. '연도' 컬럼 이름 지정
df.rename(columns={df.columns[0]: '연도'}, inplace=True)

# 4. 연도 필터링
df_filtered = df[df['연도'].isin([2017, 2018, 2019])]

# 5. melt → long → pivot으로 wide-format 재구성
지역_컬럼 = [col for col in df_filtered.columns if col != '연도']

df_melted = df_filtered.melt(
    id_vars='연도',
    value_vars=지역_컬럼,
    var_name='지역',
    value_name='총처분가능소득(십억)'
)

# 6. 숫자 전처리: 공백·쉼표 제거 → float
df_melted['총처분가능소득(십억)'] = (
    df_melted['총처분가능소득(십억)']
    .astype(str)
    .str.replace(',', '')
    .str.replace(' ', '')
    .astype(float)
)

#print(df_melted)
# 1. Unnamed로 시작하는 지역 제거 (처음 3개 행)
cond1 = df_melted['지역'].str.startswith('Unnamed')

# 2. 지역 == '전국' 제거 (다음 3개 행)
cond2 = df_melted['지역'] == '전국'

# 3. 지역 == '제주' AND 총처분가능소득이 NaN이거나 연도와 값이 같은 행 제거 (마지막 6개)
cond3 = ((df_melted['지역'] == '제주') &
         ((df_melted['총처분가능소득(십억)'].isna()) |
          (df_melted['연도'] == df_melted['총처분가능소득(십억)'])))

# 최종 필터링
df_melted_cleaned = df_melted[~(cond1 | cond2 | cond3)].reset_index(drop=True)

# 결과 확인
#print(df_melted_cleaned)

# 7. wide-format 변환
df_income = df_melted_cleaned.pivot(index='지역', columns='연도', values='총처분가능소득(십억)').reset_index()
df_income.columns.name = None

# 8. 컬럼명 지정
df_income.rename(columns={
    2017: '2017_총처분가능소득(십억)',
    2018: '2018_총처분가능소득(십억)',
    2019: '2019_총처분가능소득(십억)',
}, inplace=True)

# ✅ 결과 확인
print(df_income)


In [ ]:
import pandas as pd

# 1. 엑셀 불러오기 (다중 헤더: 2줄)
df = pd.read_excel("행정구역_시군구_별__성별_인구수_20250729122209.xlsx", header=[0, 1])

# 2. '행정구역' 열 이름을 '지역'으로 지정
columns = df.columns.tolist()
columns[0] = '지역'
df.columns = columns

# 3. '전국' 행 제거
df = df[df['지역'] != '전국'].reset_index(drop=True)

# 4. 각 연도별 총인구 컬럼만 추출
df_population = df[['지역', ('2017', '총인구수 (명)'), ('2018', '총인구수 (명)'), ('2019', '총인구수 (명)')]].copy()

# 5. 컬럼명 간단히 수정
df_population.columns = ['지역', '2017_총인구', '2018_총인구', '2019_총인구']

# 6. 지역명 단순화 매핑
지역_이름_매핑 = {
    '서울특별시': '서울', '부산광역시': '부산', '대구광역시': '대구', '인천광역시': '인천', 
    '광주광역시': '광주', '대전광역시': '대전', '울산광역시': '울산', '세종특별자치시': '세종',
    '경기도': '경기', '강원특별자치도': '강원', '충청북도': '충북', '충청남도': '충남',
    '전북특별자치도': '전북', '전라남도': '전남', '경상북도': '경북', '경상남도': '경남',
    '제주특별자치도': '제주'
}

df_population['지역'] = df_population['지역'].replace(지역_이름_매핑)

# 7. 숫자형으로 변환 (콤마 제거)
for col in ['2017_총인구', '2018_총인구', '2019_총인구']:
    df_population[col] = df_population[col].astype(str).str.replace(",", "").astype(int)

# 결과 확인
print(df_population)


In [ ]:
import pandas as pd

# 1. 지역 기준 병합
df_income_merged = pd.merge(df_income, df_population, on='지역', how='inner')

# 2. 연도 리스트
years = ['2017', '2018', '2019']

# 3. 연도별 1인당 총처분가능소득(천원) 계산
for year in years:
    income_col = f'{year}_총처분가능소득(십억)'
    pop_col = f'{year}_총인구'
    per_capita_col = f'{year}_1인당총처분가능소득(천원)'
    
    df_income_merged[per_capita_col] = (
        df_income_merged[income_col] * 1_000_000 / df_income_merged[pop_col]
    ).round()

# 4. 결과 컬럼 출력
cols_to_show = ['지역'] + [f'{year}_1인당총처분가능소득(천원)' for year in years]
print(df_income_merged[cols_to_show])


### 시군구 비중 구하기

In [ ]:
# 1. 엑셀 불러오기
df_kcb = pd.read_csv('KCB_SIGNGU_DATA5_23_202505.csv')

# 2. 시도명 추출 (시군구명에서 앞 단어)
df_kcb['시도'] = df_kcb['SIGNGU_NM'].str.split().str[0]

print(df_kcb['시도'].unique())


# 3. 결과 저장용 리스트
result_list = []

# 4. 시도별 루프
for sido, group in df_kcb.groupby('시도'):
    avg_income = group['AVRG_INCOME_PRICE'].mean()
    group = group.copy()
    group['시군구_비중(%)'] = (group['AVRG_INCOME_PRICE'] / avg_income * 100).round(2)
    result_list.append(group)

# 5. 결과 통합
df_result = pd.concat(result_list, ignore_index=True)

# 6. 필요한 열만 보기
print(df_result[['시도', 'SIGNGU_NM', 'AVRG_INCOME_PRICE', '시군구_비중(%)']])

print(df_result[df_result['시도'] == '없음(세종시)'])


### 시군구 비중을 시도별 연소득 데이터에 적용

In [ ]:
import pandas as pd

# 1. wide → long 변환
df_income_long = df_income_merged.melt(
    id_vars='지역',
    value_vars=[
        '2017_1인당총처분가능소득(천원)',
        '2018_1인당총처분가능소득(천원)',
        '2019_1인당총처분가능소득(천원)'
    ],
    var_name='연도_raw',
    value_name='시도별_1인당_총소득(천원)'
)

# 2. 연도 추출
df_income_long['연도'] = df_income_long['연도_raw'].str.extract(r'(\d{4})').astype(int)

# 3. 컬럼명 정리
df_income_long.rename(columns={'지역': '시도'}, inplace=True)

# 4. 최종 컬럼 정리
df_income_long = df_income_long[['시도', '연도', '시도별_1인당_총소득(천원)']]

#print(df_income_long)

################

# 1. 시도 이름 매핑 (df_result → df_income_long)
sido_name_map = {
    '서울특별시': '서울', '부산광역시': '부산', '대구광역시': '대구', '인천광역시': '인천',
    '광주광역시': '광주', '대전광역시': '대전', '울산광역시': '울산', '없음(세종시)': '세종',
    '경기도': '경기', '강원특별자치도': '강원', '충청북도': '충북', '충청남도': '충남',
    '전북특별자치도': '전북', '전라남도': '전남', '경상북도': '경북', '경상남도': '경남',
    '제주특별자치도': '제주'
}
df_result['시도'] = df_result['시도'].map(sido_name_map)

print(df_result['시도'].unique())

# 2. 연도별 반복 적용
df_result_all = []

for year in [2017, 2018, 2019]:
    df_year = df_result.copy()
    df_year['연도'] = year

    df_year = pd.merge(
        df_year,
        df_income_long[df_income_long['연도'] == year],
        on='시도',
        how='left',
        suffixes=('', '_drop')  # 충돌 방지용 suffix
    )

    df_year = df_year.drop(columns=[col for col in df_year.columns if '연도' in col and col != '연도'])

    df_year['시군구_1인당_총소득(천원)'] = (
        df_year['시도별_1인당_총소득(천원)'] * df_year['시군구_비중(%)'] / 100
    ).round(1)

    df_result_all.append(df_year)

# 3. 결과 통합
df_result_final = pd.concat(df_result_all, ignore_index=True)
print(df_result_final.columns.tolist())


# 4. 열 정리
cols_order = [
    '연도', '시도', 'SIGNGU_NM', 'AVRG_INCOME_PRICE',
    '시군구_비중(%)', '시도별_1인당_총소득(천원)', '시군구_1인당_총소득(천원)'
]
df_result_final = df_result_final[cols_order]

# 5. 결과 확인
print(df_result_final.head())


### 연소득 -> 월소득 

In [ ]:
import pandas as pd
import numpy as np


# 1. '연도'를 연도의 중앙값인 7월 1일로 매핑하여 datetime 형식의 인덱스 생성
df_result_final['일자'] = pd.to_datetime(df_result_final['연도'].astype(str) + '-07-01')

# 시군구별로 데이터를 그룹화하고 보간 수행
interpolated_dfs = []

for sigungu, group in df_result_final.groupby(['시도', 'SIGNGU_NM']):
    # 연도별 데이터를 datetime 인덱스로 설정하고 정렬
    temp = group.sort_values('일자').set_index('일자')

    # 보간할 전체 월별 인덱스 생성 (최소/최대 연도에서 넉넉하게 확장)
    # 예를 들어, 2017년 데이터가 있다면 2017년 1월부터 시작하여 보간 가능성을 높입니다.
    full_index = pd.date_range(
        start=temp.index.min().to_period('Y').start_time, # 해당 연도의 1월 1일
        end=temp.index.max().to_period('Y').end_time,   # 해당 연도의 12월 말일
        freq='MS' # 월의 시작일
    )

    # 전체 월별 인덱스에 기존 데이터를 재색인 (중간에 NaN이 생김)
    temp_reindexed = temp.reindex(full_index)

    # '시도'와 'SIGNGU_NM' 같은 식별 정보는 NaN으로 채워지므로 다시 채워줍니다.
    temp_reindexed['시도'] = sigungu[0]
    temp_reindexed['SIGNGU_NM'] = sigungu[1]

    # '시군구_1인당_총소득(천원)' 열만 선형 보간 수행
    # interpolate(method='linear')는 기존 데이터 포인트 사이의 NaN만 채웁니다.
    # 만약 그룹 내에 유효한 데이터 포인트가 1개만 있다면 보간은 일어나지 않습니다.
    temp_reindexed['시군구_1인당_총소득(천원)'] = temp_reindexed['시군구_1인당_총소득(천원)'].interpolate(method='linear')

    # 연/월 추출
    temp_reindexed['연도'] = temp_reindexed.index.year
    temp_reindexed['월'] = temp_reindexed.index.month

    # 'AVRG_INCOME_PRICE'는 사용하지 않으므로 포함하지 않습니다.
    # '일자', '시군구_비중(%)', '시도별_1인당_총소득(천원)' 등 원본에 있던 다른 열들은
    # 필요에 따라 포함하거나 제외할 수 있습니다. 여기서는 결과물에 필요한 열만 선택합니다.

    interpolated_dfs.append(temp_reindexed.reset_index(drop=True))

# 전체 보간된 데이터프레임 병합
df_monthly = pd.concat(interpolated_dfs, ignore_index=True)

# 최종 결과물에 필요한 컬럼만 선택하고 순서 정리
df_monthly = df_monthly[[
    '연도', '월', '시도', 'SIGNGU_NM', '시군구_1인당_총소득(천원)'
]]

# print(df_monthly)

# 요청하신 2017년 9월부터 2019년 9월까지의 자료만 필터링
# 시작 연도와 끝 연도의 월을 정확히 지정하여 필터링합니다.
df_monthly_filtered = df_monthly[
    # 2017년의 경우 9월부터 포함
    ((df_monthly['연도'] == 2017) & (df_monthly['월'] >= 9)) |
    # 2018년은 모든 월 포함
    ((df_monthly['연도'] == 2018)) |
    # 2019년의 경우 9월까지 포함
    ((df_monthly['연도'] == 2019) & (df_monthly['월'] <= 9))
].copy()


In [ ]:
# --- 로그 변환을 적용하는 부분 ---
# '시군구_1인당_총소득(천원)' 컬럼에 np.log() 함수 적용
# 값이 0이거나 음수인 경우 오류가 발생할 수 있으므로, 실제 데이터에 음수나 0이 없는지 확인 필요
df_monthly_filtered['시군구_1인당_총소득(천원)_log'] = np.log(df_monthly_filtered['시군구_1인당_총소득(천원)'])

# 기존 컬럼을 로그 변환된 값으로 대체하거나, 새로운 컬럼으로 추가할 수 있습니다.
# 여기서는 새로운 컬럼으로 추가하고, 기존 컬럼은 그대로 둡니다.
# 만약 기존 컬럼을 대체하고 싶다면, 아래와 같이 사용하세요:
# df_monthly_filtered['시군구_1인당_총소득(천원)'] = np.log(df_monthly_filtered['시군구_1인당_총소득(천원)'])
# -----------------------------

# 결과 예시 확인 (필터링된 데이터와 로그 변환된 컬럼 포함)
print(df_monthly_filtered.head(20))

In [ ]:
# --------------- 마지막 세부 전처리 
# '시도'가 '세종'인 행의 'SIGUNGU_NM' 값 수정
# ——— 1) 세종시명 특별 처리 ———
df_monthly_filtered.loc[
    df_monthly_filtered['시도'] == '세종',
    'SIGNGU_NM'
] = '세종특별자치시 세종시'

# ——— 2) 연도+월 → 시간(YYYY-MM) 생성 ———
df_monthly_filtered['시간'] = (
    df_monthly_filtered['연도'].astype(int).astype(str)
    + '-'
    + df_monthly_filtered['월']
        .astype(int)
        .map(lambda m: f"{m:02d}")
)

# ——— 3) 더 이상 필요 없는 연도·월 컬럼 삭제 ———
df_monthly_filtered.drop(
    columns=['연도', '월'],
    inplace=True
)

# ——— 4) (혹시 이전에 잘못 생성된) '시군구' 컬럼 있으면 제거 ———
if '시군구' in df_monthly_filtered.columns:
    df_monthly_filtered.drop(columns=['시군구'], inplace=True)

# ——— 5) SIGNGU_NM 이름만 '시군구'로 바꾸기 ———
df_monthly_filtered.rename(
    columns={'SIGNGU_NM': '시군구'},
    inplace=True
)

# ——— 결과 확인 ———
print(df_monthly_filtered.head())

# ——— (선택) 저장 ———
df_monthly_filtered.to_excel(
    '월별_시군구_1인당_총소득_로그변환_최종.xlsx',
    index=False
)



# 엑셀 파일로 저장
#output_file_name = '월별_시군구_1인당_총소득_로그변환.xlsx'
#df_monthly_filtered.to_excel(output_file_name, index=False)

#print(f"'{output_file_name}' 파일이 성공적으로 저장되었습니다.")
# 없음(세종시) -> 세종특별자치시 세종시 엑셀로 수동변환.

### 이후 df merge할 때 고려할 도시들


경기도 부천시 소사구, 경기도 부천시 오정구, 경기도 부천시 원미구 -> 부천시로 통합해야함.

In [ ]:
df_monthly_filtered = pd.read_excel("월별_시군구_1인당_총소득_로그변환_최종.xlsx")

df_monthly_filtered.head()

In [ ]:
import numpy as np
import pandas as pd

# 1) 합칠 부천시 하위 구 리스트
bucheon_subs = [
    '경기도 부천시 소사구',
    '경기도 부천시 오정구',
    '경기도 부천시 원미구'
]

# 2) 해당 행만 골라내는 마스크
mask = df_monthly_filtered['시군구'].isin(bucheon_subs)

# 3) 시간별 평균 계산
bucheon_avg = (
    df_monthly_filtered[mask]
    .groupby('시간', as_index=False)['시군구_1인당_총소득(천원)']
    .mean()
)

# 4) 새 행에 들어갈 컬럼 값 세팅
bucheon_avg['시군구'] = '경기도 부천시'
bucheon_avg['시도']   = '경기'
# 평균값에 로그 씌워서 _log 컬럼 생성
bucheon_avg['시군구_1인당_총소득(천원)_log'] = np.log(
    bucheon_avg['시군구_1인당_총소득(천원)']
)

# 5) 원래 컬럼 순서로 재정렬
bucheon_avg = bucheon_avg[
    ['시도','시군구','시군구_1인당_총소득(천원)','시군구_1인당_총소득(천원)_log','시간']
]

# 6) 원본에서 소사·오정·원미구 행 삭제
df_monthly_filtered = df_monthly_filtered[~mask]

# 7) 평균값 행 추가
df_monthly_filtered = pd.concat(
    [df_monthly_filtered, bucheon_avg],
    ignore_index=True
)

# (선택) 보기 편하게 정렬
df_monthly_filtered.sort_values(
    by=['시도','시군구','시간'],
    inplace=True,
    ignore_index=True
)

# 결과 확인
print(df_monthly_filtered.tail(10))


In [27]:
df_monthly_filtered.to_excel("log_income.xlsx", index=False)

# log pop 전처리

https://kosis.kr/statHtml/statHtml.do?orgId=101&tblId=DT_1B040A3&conn_path=I2

행정안전부,「주민등록인구현황」, 2025.07, 2025.08.12, 행정구역(시군구)별, 성별 인구수

-> 그냥 형식 맞추어 정리하는 정도.

In [ ]:
import pandas as pd

# CSV 또는 Excel 불러오기
df = pd.read_excel("시군구별_인구수.xlsx")  # 또는 pd.read_excel()

# 결과를 담을 리스트
current_province = None
new_rows = []

# 시군구 컬럼 이름 가져오기
columns = df.columns

# row별로 순회
for _, row in df.iterrows():
    region_raw = row.iloc[0]  # 첫 열: 행정구역명
    region_str = str(region_raw)

    if region_str.startswith("　") or region_str.startswith(" "):  # 전각 공백 또는 일반 공백
        # 시군구 → 시도 붙여서 이름 생성
        region_name = region_str.strip()  # 앞뒤 공백 제거
        full_region_name = f"{current_province} {region_name}"
        new_row = [full_region_name] + list(row.iloc[1:])
        new_rows.append(new_row)
    else:
        # 들여쓰기 없는 경우 → 시도명으로 간주
        current_province = region_str.strip()

# 새로운 데이터프레임 생성
new_df = pd.DataFrame(new_rows, columns=columns)

# 결과 확인
#print(new_df.head())

provincelist = new_df["행정구역(시군구)별"].unique()
print(len(provincelist))
print(provincelist)


In [ ]:
# 두 리스트를 set으로 변환
SIGNGU_NM_list = df_monthly_filtered["시군구"].unique()

set1 = set(SIGNGU_NM_list)
set2 = set(provincelist)

# 서로 다른 값 확인
only_in_set1 = set1 - set2
only_in_set2 = set2 - set1

print("✅ SIGNGU_NM_list에만 있고 provincelist에는 없는 시군구들:")
print(sorted(only_in_set1))

print("\n✅ provincelist에만 있고 SIGNGU_NM_list에는 없는 시군구들:")
print(sorted(only_in_set2))


아 ... 252개가 맞겠다. (ex. 고양시 안에 덕양구, 일산동구, 일산서구가 있는데 province는 고양시, 덕양구, 이렇게 봄)

-> 262개 좀 더 정리해볼 수는 있겠는데 ... 흠흠흠

In [31]:
problematic_mapping = {
    '경기도 덕양구': '경기도 고양시 덕양구',
    '경기도 일산동구': '경기도 고양시 일산동구',
    '경기도 일산서구': '경기도 고양시 일산서구',
    '경기도 소사구': '경기도 부천시 소사구',
    '경기도 오정구': '경기도 부천시 오정구',
    '경기도 원미구': '경기도 부천시 원미구',
    '경기도 분당구': '경기도 성남시 분당구',
    '경기도 수정구': '경기도 성남시 수정구',
    '경기도 중원구': '경기도 성남시 중원구',
    '경기도 권선구': '경기도 수원시 권선구',
    '경기도 영통구': '경기도 수원시 영통구',
    '경기도 장안구': '경기도 수원시 장안구',
    '경기도 팔달구': '경기도 수원시 팔달구',
    '경기도 단원구': '경기도 안산시 단원구',
    '경기도 상록구': '경기도 안산시 상록구',
    '경기도 동안구': '경기도 안양시 동안구',
    '경기도 만안구': '경기도 안양시 만안구',
    '경기도 기흥구': '경기도 용인시 기흥구',
    '경기도 수지구': '경기도 용인시 수지구',
    '경기도 처인구': '경기도 용인시 처인구',
    '경상남도 마산합포구': '경상남도 창원시 마산합포구',
    '경상남도 마산회원구': '경상남도 창원시 마산회원구',
    '경상남도 성산구': '경상남도 창원시 성산구',
    '경상남도 의창구': '경상남도 창원시 의창구',
    '경상남도 진해구': '경상남도 창원시 진해구',
    '경상북도 남구': '경상북도 포항시 남구',
    '경상북도 북구': '경상북도 포항시 북구',
    '경상북도 군위군': '대구광역시 군위군',
    '세종특별자치시 세종시': '세종특별자치시 세종시',
    '전북특별자치도 덕진구': '전북특별자치도 전주시 덕진구',
    '전북특별자치도 완산구': '전북특별자치도 전주시 완산구',
    '충청남도 동남구': '충청남도 천안시 동남구',
    '충청남도 서북구': '충청남도 천안시 서북구',
    '충청북도 상당구': '충청북도 청주시 상당구',
    '충청북도 서원구': '충청북도 청주시 서원구',
    '충청북도 청원구': '충청북도 청주시 청원구',
    '충청북도 흥덕구': '충청북도 청주시 흥덕구',
}


In [32]:
def normalize_region_exact(name):
    # 부천시는 예외적으로 그대로 사용
    if name == '경기도 부천시':
        return '경기도 부천시'

    # 제거 대상: 부천시의 옛 구들
    if name in ['경기도 소사구', '경기도 오정구', '경기도 원미구']:
        return None

    # 일반적인 매핑 처리
    if name in problematic_mapping:
        return problematic_mapping[name]
    
    # 시만 있는 경우 (부천시 제외): 제거
    if name in [
        '경기도 고양시', '경기도 성남시', '경기도 수원시', '경기도 안산시',
        '경기도 안양시', '경기도 용인시', '경상남도 창원시', '경상북도 포항시',
        '전북특별자치도 전주시', '충청남도 천안시', '충청북도 청주시'
    ]:
        return None

    # 나머지 그대로
    return name


In [ ]:
import pandas as pd

# 예시 데이터

# 정제 적용
new_df['시군구_정제'] = new_df.iloc[:, 0].apply(normalize_region_exact)
# 결과 확인
print(new_df)




In [ ]:
#new_df.to_excel("월별_시군구_인구수_예비.xlsx", index=False)

In [ ]:
# NaN 제거
new_df_clean = new_df.dropna(subset=['시군구_정제'])

# 유니크 개수 출력
print("정제된 시군구 개수:", new_df_clean['시군구_정제'].nunique())

output_file_name = '월별_시군구_인구수_로그변환.xlsx'
new_df_clean.to_excel(output_file_name, index=False)


In [ ]:
print(new_df_clean)

In [ ]:
# 두 리스트를 set으로 변환
sigungu_cleaned = new_df_clean["시군구_정제"].unique()
set1 = set(SIGNGU_NM_list)
set2 = set(sigungu_cleaned )

# 서로 다른 값 확인
only_in_set1 = set1 - set2
only_in_set2 = set2 - set1

print("✅ SIGNGU_NM_list에만 있고 sigungu_cleaned 에는 없는 시군구들:")
print(sorted(only_in_set1))

print("\n✅sigungu_cleaned 에만 있고 SIGNGU_NM_list에는 없는 시군구들:")
print(sorted(only_in_set2))


sigungu_cleaned(인구수) 기준으로 맞추기 !!!!!

이후 SIGUNGU_NM(income)에서도 경기도 부천시 소사구, 경기도 부천시 오정구, 경기도 부천시 원미구 -> 부천시로 통합해야함.

추후 df_psl (시군구 249개)에 맞추기 위해 '경상남도 창원시 성산구' + '경상남도 창원시 의창구' -> '경상남도 창원시'

In [ ]:
import pandas as pd
import numpy as np

# 1) 엑셀 파일 불러오기
#new_df_clean = pd.read_excel('/mnt/data/월별_시군구_인구수_로그변환.xlsx')

# 2) 불필요한 컬럼 삭제 및 컬럼명 변경
new_df_clean = new_df_clean.drop(columns=['행정구역(시군구)별'])
new_df_clean = new_df_clean.rename(columns={'시군구_정제': '시군구'})

# 3) '시도' 컬럼 생성: 시군구 문자열에서 맨 앞의 행정구역명(예: '서울특별시')을 추출하여 매핑
sido_name_map = {
    '서울특별시': '서울', '부산광역시': '부산', '대구광역시': '대구', '인천광역시': '인천',
    '광주광역시': '광주', '대전광역시': '대전', '울산광역시': '울산', '세종특별자치시': '세종',
    '경기도': '경기', '강원특별자치도': '강원', '충청북도': '충북', '충청남도': '충남',
    '전북특별자치도': '전북', '전라남도': '전남', '경상북도': '경북', '경상남도': '경남',
    '제주특별자치도': '제주'
}
# 시군구 컬럼의 첫 번째 단어(공백 앞까지)를 뽑아서 매핑
new_df_clean['시도'] = (
    new_df_clean['시군구']
        .str.split().str[0]
        .map(sido_name_map)
)

# 4) wide → long 포맷 변환 (2017.09 ~ 2019.09 컬럼을 '시간'·'pop'으로)
date_cols = [c for c in new_df_clean.columns if c not in ['시군구', '시도']]
df_long = new_df_clean.melt(
    id_vars=['시도', '시군구'],
    value_vars=date_cols,
    var_name='시간',
    value_name='pop'
)

# 5) '시간' 컬럼 포맷 수정: 'YYYY.MM' → 'YYYY-MM'
df_long['시간'] = df_long['시간'].str.replace('.', '-', regex=False)

# 6) 인구수에 로그 변환 후 'log_pop' 컬럼으로 저장
df_long['log_pop'] = np.log(df_long['pop'])

# 7) 최종 컬럼 순서대로 선택
df_result = df_long[['시간', '시도', '시군구', 'log_pop']]

# 8) (선택) 결과 확인
print(df_result.head())

# 9) (선택) 파일로 저장
df_result.to_excel(
    '월별_시군구_인구수_로그변환_최종.xlsx',
    index=False
)


In [ ]:
print(len(df_result['시군구'].unique()))
len(df_monthly_filtered['시군구'].unique())

### 시군구 통합 시 고려해야할 것 (결론,,)

df_psl (시군구 249개) 기준으로 SIGUNGU_NM(income), sigungu_cleaned(인구수)맞추기 !!!!!

1.  SIGUNGU_NM(income)

 경기도 부천시 소사구, 경기도 부천시 오정구, 경기도 부천시 원미구 -> 부천시로 통합해야함. -> 완료
 
 추후 df_psl (시군구 249개)에 맞추기 위해 '경상남도 창원시 성산구' + '경상남도 창원시 의창구' -> '경상남도 창원시'
 
 


2. sigungu_cleaned(인구수)

'경상남도 창원시 성산구' + '경상남도 창원시 의창구' -> '경상남도 창원시'

# user cost 전처리

1. 기준금리: 코픽스(COFIX) – 주택담보대출의 기준금리
(국내 은행의 자금조달 비용을 반영한 주택담보대출 기준금리 → 일반 소비자가 체감하는 실제 주담대 금리에 가장 가까움)
전국은행연합회 소비자포털
: 신규취급액 기준 COFIX(최근 신규대출에 적용된 평균 자금조달 비용이므로 시장금리의 단기 변동을 빠르게 반영함)

2. 주택가격상승률(전년동기대비) : (현재 매매가격지수/작년 매매가격지수) -1
→ log_price 값 활용하면 될 듯? (2016-09 ~) 자료가 필요함..

3. 감가상각률 : 주택이라는 자산이 시간에 따라 가치가 떨어지는 속도(보유만으로도 생기는 비용) → 0.02 고정값 사용 예정

국내논문/정책실무(국토부,KDI)에서도 상수 처리함



### COFIX 전처리

In [ ]:
import pandas as pd

# 1) 각 연도별 COFIX 통계 엑셀 읽기
cofix1 = pd.read_excel('COFIX통계(2017년도)_20250801.xlsx', skiprows=1, header=0, usecols=['대상월', '신규취급액기준 COFIX'])
cofix2 = pd.read_excel('COFIX통계(2018년도)_20250801.xlsx', skiprows=1, header=0, usecols=['대상월', '신규취급액기준 COFIX'])
cofix3 = pd.read_excel('COFIX통계(2019년도)_20250801.xlsx', skiprows=1, header=0, usecols=['대상월', '신규취급액기준 COFIX'])

# 2) 세 년치 합치기
cofix = pd.concat([cofix1, cofix2, cofix3], ignore_index=True)

# 3) 컬럼명 바꾸기
cofix.rename(
    columns={
        '대상월': '시간',
        '신규취급액기준 COFIX': 'cofix_new'
    },
    inplace=True
)

# 4) '시간' 포맷: 'YYYY/MM' → 'YYYY-MM'
cofix['시간'] = (
    cofix['시간']
         .astype(str)
         .str.replace('/', '-', regex=False)
         .str.split('-')
         .apply(lambda x: f"{x[0]}-{int(x[1]):02d}")
)

# 5) 2017-09 ~ 2019-09 필터링
cofix = cofix[(cofix['시간'] >= '2017-09') & (cofix['시간'] <= '2019-09')]

# 6) 인덱스 리셋
cofix.reset_index(drop=True, inplace=True)

#print(cofix)

# (선택) 파일로 저장
#df.to_excel('/COFIX.xlsx', index=False)

# 이미 '시간'이 'YYYY-MM' 문자열인 df라면, datetime으로 변환 후 정렬하는 것이 안전합니다:
cofix['시간'] = pd.to_datetime(cofix['시간'], format='%Y-%m')

# 오름차순(2017-09 → 2019-09)으로 정렬
cofix = cofix.sort_values('시간').reset_index(drop=True)

# 다시 'YYYY-MM' 문자열로 바꾸고 완료
cofix['시간'] = cofix['시간'].dt.strftime('%Y-%m')

print(cofix)

### 증감율 데이터와 통합하기

재원오빠가 정리해준 주택매매가격 증감율 데이터에다 cofix 합치기

In [ ]:
df_fluc = pd.read_excel("증감율 전처리 최종.xlsx")

len(df_fluc["시군구"].unique())

In [ ]:
# '시간' 컬럼을 통일된 문자열 포맷으로 맞추기 (혹시 모를 형식 불일치 방지)
df_fluc['시간']    = pd.to_datetime(df_fluc['시간'],    format='%Y-%m').dt.strftime('%Y-%m')
cofix['시간'] = pd.to_datetime(cofix['시간'], format='%Y-%m').dt.strftime('%Y-%m')

# '시간'을 키로 left join
df_fluc_cofix_merged = pd.merge(
    df_fluc,
    cofix[['시간', 'cofix_new']],
    on='시간',
    how='left'      # df_fluc 기준으로, cofix 없으면 NaN
)

# 5) (선택) 결과 확인
print(df_fluc_cofix_merged)

# 6) (선택) 파일로 저장
#df_merged.to_excel(
    #'/mnt/data/증감율_전처리_최종_cofix_병합.xlsx',
    #index=False
#)


In [ ]:
# 사용자비용 계산:
# 금리(cofix_new) – (주택가격상승률 = 증감율 – 1) + 감가상각률(0.02)
df_fluc_cofix_merged['사용자비용'] = (
    df_fluc_cofix_merged['cofix_new']
    - (df_fluc_cofix_merged['증감율'] - 1)
    + 0.02
)



In [ ]:
# 세종특별자치시 특별 처리
print(df_fluc_cofix_merged[['시군구','시간','증감율','cofix_new','사용자비용']].head())

df_fluc_cofix_merged.loc[
    df_fluc_cofix_merged['시군구'] == '세종특별시 세종',
    '시군구'
] = '세종특별자치시 세종시'

In [77]:
usercost = df_fluc_cofix_merged[['시군구','시간','사용자비용']]
usercost.to_excel(
    'usercost.xlsx',
    index=False
)

In [ ]:
len(usercost['시군구'].unique())

# log_income + log_pop + usercost

In [ ]:
# Assuming the following DataFrames are already loaded:
# - usercost:       ['시간', '시군구', '사용자비용']
# - df_monthly_filtered: ['시간', '시군구', '시군구_1인당_총소득(천원)_log']
# - df_result:      ['시간', '시군구', 'log_pop']

# 1) usercost + income-log
merged = usercost.merge(
    df_monthly_filtered[['시간', '시군구', '시군구_1인당_총소득(천원)_log']],
    on=['시간', '시군구'],
    how='inner'
)

# 2) + population-log
merged = merged.merge(
    df_result[['시간', '시군구', 'log_pop']],
    on=['시간', '시군구'],
    how='inner'
)

# 3) 최종 컬럼 선택 및 인덱스 리셋
final_df = merged[
    ['시간', '시군구', '시군구_1인당_총소득(천원)_log', 'log_pop', '사용자비용']
].reset_index(drop=True)

# 4) 결과 확인
print(final_df.head())

# (Optional) 저장
final_df.to_excel('log_income+log_pop+usercost.xlsx', index=False)


일단 끝내고,,, 보니 .. 2개 결측치 확인 됐는데.. 걍 버리자

In [ ]:
len(final_df['시군구'].unique())

In [ ]:
len(merged['시군구'].unique())

In [ ]:
before = set(usercost['시군구'])
after  = set(merged['시군구'])  # usercost + income-log 병합한 DataFrame
print("병합 전 시군구 수:", len(before))
print("병합 후 시군구 수:", len(after))

missing = before - after
print("병합에서 빠진 시군구:", missing)


그냥 .. 이제 모르겠다 충북, 청주 버려....

# log_price,land,stock과 merge

In [ ]:
import pandas as pd

# 1. CSV 파일 로드
df_price = pd.read_csv('log_price_증감률포함.csv', encoding='utf-8')

# 2. 컬럼명 확인 및 통일
print("Original columns:", df_price.columns.tolist())
if '지역' in df_price.columns:
    df_price = df_price.rename(columns={'지역': '시군구'})
    
# 3. '시간' 문자열을 datetime으로 파싱 (예: 'Sep-16' → 2016-09-01)
df_price['시간'] = pd.to_datetime(df_price['시간'], format='%b-%y')

# 4. 2017-09 이상의 데이터만 필터링
df_price = df_price[df_price['시간'] >= '2017-09-01'].copy()

# 5. 다시 'YYYY-MM' 형식의 문자열로 변환
df_price['시간'] = df_price['시간'].dt.strftime('%Y-%m')

# 6. 결과 확인
print(df_price.head())
print(f"Filtered date range: {df_price['시간'].min()} ~ {df_price['시간'].max()}")



In [108]:
df_price['ln_stock'] = np.log(df_price['log_stock'])

In [ ]:
# 1) log_land → 자연로그
#df_price['ln_land'] = np.log(df_price['log_land'])

# 2) 최종 테이블 구성
df_pls = (
    df_price[['시군구', '시간', 'ln_price', 'ln_land', 'ln_stock']]
    .copy()
)

# 3) 확인
print(df_pls.head())
print(df_pls.shape)  # (행 개수, 5)


In [ ]:
# 1) ln_price가 NaN인 시군구 목록
bad_regions = df_pls[df_pls['ln_price'].isna()]['시군구'].unique()

# 2) 해당 시군구를 제외하고 새 데이터프레임 생성
df_pls_clean = df_pls[~df_pls['시군구'].isin(bad_regions)].copy()

# 3) 확인
print("제거된 시군구 수:", len(bad_regions))
print("정제 후 크기:", df_pls_clean.shape)

# 세종특별자치시 특별 처리
df_pls_clean['시군구'] = df_pls_clean['시군구'].replace('세종특별자치시 세종', '세종특별자치시 세종시')


In [118]:
df_pls_clean.to_excel("log_price+log_land+log_stock.xlsx", index=False)

### 두 엑셀 간 시군구 비교

In [ ]:
import pandas as pd

# 1. 두 개의 엑셀 파일(CSV)을 각각 불러옵니다.
df1 = pd.read_excel("log_price+log_land+log_stock.xlsx")
df2 = pd.read_excel("log_income+log_pop+usercost.xlsx")

# 2. 각 데이터프레임에서 '시군구' 열의 고유한 값들을 집합(set)으로 만듭니다.
# 집합으로 만들면 중복된 값이 사라지고 비교하기 용이해집니다.
sigungu_set1 = set(df1['시군구'].unique())
sigungu_set2 = set(df2['시군구'].unique())

print(len(df1['시군구'].unique()))
print(len(df2['시군구'].unique()))

# 3. 두 집합을 비교하여 차집합을 구합니다.
# 첫 번째 파일에만 있는 시군구
diff1 = sigungu_set1 - sigungu_set2
# 두 번째 파일에만 있는 시군구
diff2 = sigungu_set2 - sigungu_set1

# 4. 결과를 출력합니다.
print("--- 첫 번째 파일에만 존재하는 시군구 목록 ---")
# 보기 좋게 정렬하여 출력합니다.
print(sorted(list(diff1)))

print("\n--- 두 번째 파일에만 존재하는 시군구 목록 ---")
print(sorted(list(diff2)))

이제 그냥 174개 시군구로 확정... 

### 이제 진짜 merge

In [ ]:
import pandas as pd

# 1) 두 개의 데이터 파일 불러오기
#df_price = pd.read_excel("log_price+log_land+log_stock_final.xlsx") -> df1
#df_income = pd.read_csv("log_income+log_pop+usercost.xlsx - Sheet1.csv") ->df2

# 2) '시간'과 '시군구'를 기준으로 두 데이터프레임 병합 (merge)
df_final = pd.merge(df1, df2, on=['시간', '시군구'])

# 3) 병합된 데이터프레임 정보 확인
print("--- 병합 후 최종 데이터 정보 ---")
print(len(df_final['시군구'].unique()))
print("\n--- 최종 데이터 샘플 ---")
print(df_final.head())
nan_counts = df_final.isnull().sum()

# 3) 결과 출력
print("--- 각 열의 NaN 값 개수 ---")
print(nan_counts)

# 4) 최종 데이터셋을 새로운 엑셀 파일로 저장
#df_final.to_excel("final_dataset.xlsx", index=False)

### post_policy, treated_region, policy_effect 변수 만들기


2018년 9월 13일자 “투기과열지구”에 해당하는 46개 시군구 이름 (결국 174개 중 46개구가 treated_region)


2018년 9월 13일부 「9·13 주택시장 안정화 대책(공고 제2018‑2297호)」 기준으로,
국토교통부 정책 발표 문서·이후 경과 기록(연혁)·뉴스 등에 명시된 투기과열지구 및 조정대상지역 지정 명단을 정제한 리스트

https://www.etaxkorea.net/sub/sub_page.php?sp=s010505&md=show&code=08&m_code=1

https://u12134.tistory.com/m/21216?utm_source=chatgpt.com

In [122]:
treated_region = [
    # 경기도
    '경기도 과천시',
    '경기도 광명시',
    '경기도 구리시',
    '경기도 군포시',
    '경기도 성남시 분당구',
    '경기도 성남시 수정구',
    '경기도 안산시 단원구',
    '경기도 안양시 동안구',
    '경기도 안양시 만안구',
    '경기도 용인시 기흥구',
    '경기도 용인시 수지구',
    '경기도 의왕시',
    '경기도 하남시',

    # 대구광역시
    '대구광역시 수성구',

    # 대전광역시
    '대전광역시 동구',
    '대전광역시 서구',
    '대전광역시 유성구',
    '대전광역시 중구',

    # 서울특별시
    '서울특별시 강남구',
    '서울특별시 강동구',
    '서울특별시 강북구',
    '서울특별시 강서구',
    '서울특별시 관악구',
    '서울특별시 광진구',
    '서울특별시 구로구',
    '서울특별시 금천구',
    '서울특별시 노원구',
    '서울특별시 도봉구',
    '서울특별시 동대문구',
    '서울특별시 동작구',
    '서울특별시 마포구',
    '서울특별시 서대문구',
    '서울특별시 서초구',
    '서울특별시 성동구',
    '서울특별시 성북구',
    '서울특별시 송파구',
    '서울특별시 양천구',
    '서울특별시 영등포구',
    '서울특별시 용산구',
    '서울특별시 은평구',
    '서울특별시 종로구',
    '서울특별시 중구',
    '서울특별시 중랑구',

    # 인천광역시
    '인천광역시 남동구',
    '인천광역시 서구',
    '인천광역시 연수구'
]


In [ ]:
import pandas as pd

# treated_region 변수 생성
df_final['treated_region'] = df_final['시군구'].isin(treated_region).astype(int)

# post_policy, policy_effect 변수까지 한 번에
df_final['post_policy']   = (df_final['시간'] >= '2018-10-01').astype(int)
df_final['policy_effect'] = df_final['treated_region'] * df_final['post_policy']

df_final.head()


In [ ]:
df_final['policy_effect'].sum()

In [124]:
df_final.to_excel("final_dataset.xlsx", index=False)